In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests


from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
# -----------------------------------------------------------------------

pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.


In [ ]:
def from_csv_to_lists(csv):
  """
    Esta función toma una archivo csv y los pasa a lista de tuplas.
    
    Args:
    - csv: archivo csv.

    Returns:
    - list(df.itertuples(index=False, name=None)): lisat de tuplas
    """
  #lee el csv y lo pasa  a dataframe
  df = pd.read_csv(csv)
  try:
    df = df.drop('Unnamed: 0', axis=1)
  except:
    pass
  #devuelve lisat de tuplas
  return list(df.itertuples(index=False, name=None))

In [ ]:
#lista de prueba:
#lista_pelis = [("Drama","Movie","The Drop",2006,6,"tt0340084"),("Thriller","Short","Geocatching",2015,11,"tt5142104"),("Drama","Movie","Blade Runner 2049", 2017,10,"tt1856101"),("Mystery","Movie","The Fugitive",1993,9,"tt0106977")]

def datos_imdb(lista_pelis):
    """
    Esta función toma una lista de tuplas de peliculas (genero, tipo, nombre pelicula, año y mes de estreno, id pelicula), busca más informacion de la pelicula en imdb y crea una nueva lista de tuplas con toda la informacion incluida para cada pelicula, otra con las puntuaciones y otra con los actores.
    
    Args:
    - lista_pelis: lista de tuplas de peliculas (genero, tipo, nombre pelicula, año y mes de estreno, id pelicula) proviniente de la api MoviesDatabase.

    Returns:
    - lista_peliculas: lista de tuplas de cada pelicula con toda su informacion (id, nombre peli, tipo,	genero,	año, mes, director, actores, guionistas, argumento, duracion).
    - lista_imdb: lista de tuplas de cada peli con su puntuacion imdb (id, nombre pelicula, puntos imdb, año)
    - lista_tupla_actores: lista de tupla de los 10 actors principales de cada peli (id, actor1 - actor10)
    """

    lista_imdb =[]
    lista_tupla_actores=[]
    lista_peliculas = []
    lista_pelicula =  []

    driver = webdriver.Chrome()

    driver.get("https://www.imdb.com/")

    driver.maximize_window()


    for tupla in lista_pelis:
        

        sleep(2)
        #Escribe el título de la pelicula en el buscador
        driver.find_element("css selector", "#suggestion-search").send_keys(tupla[-1], Keys.ENTER)

        sleep(2)
        # Puntución IMDB
        try:
        
            imdb_puntos = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.jJsEuz > div.sc-3a4309f8-0.fjtZsE.sc-dffc6c81-1.fJrHDo > div > div:nth-child(1) > a > span > div > div.sc-bde20123-0.gtEgaf > div.sc-bde20123-2.gYgHoj > span.sc-bde20123-1.iZlgcd").text
        except:
            imdb_puntos = "NO DATA"
        print(imdb_puntos)

        # Director IMDB
        # está dando error al appendear: 
        # TypeError: 'builtin_function_or_method' object is not subscriptable

        try:
            imdb_director = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > div > section > div > div.sc-9178d6fe-1.kFxVZc.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.gmonkL.title-cast.title-cast--movie.celwidget > ul > li:nth-child(1) > div > ul > li > a").text
            print(imdb_director)

        except:
            imdb_director = "NO DATA"

        # Guionistas IMDB
        guionistas = []
        for i in range(1,6):
            try: 
                imdb_guion = driver.find_element("css selector", f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > div > section > div > div.sc-9178d6fe-1.kFxVZc.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.gmonkL.title-cast.title-cast--movie.celwidget > ul > li:nth-child(2) > div > ul > li:nth-child({i}) > a").text

                guionistas.append(imdb_guion)
            except:
                break
        print(guionistas)
        
            
        sleep(2)

        # Argumento IMDB
        # Es importante dejar abrir la pagina en maximize porque si no no nos lo coge el css selector.
        try:
            imdb_argumento = driver.find_element("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-4.ecgcFy > div.sc-e226b0e3-6.hfusNC > div.sc-e226b0e3-10.jcOwsU > section > p > span.sc-466bb6c-2.eVLpWt').text
            print(imdb_argumento)
        except:
            imdb_argumento = "NO DATA"    


        # Minutos IMDB
        try:
            imdb_minutos = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.jJsEuz > div.sc-dffc6c81-0.iwmAVw > ul > li:nth-child(3)").text

            lista_min = imdb_minutos.split()
        
            #print(lista_min)
            try: 
                #Ponemos un try...except porque nos hemos dado cuenta de que algunas películas tienen otra característica en la poscición de "hora,minutos"
                # Por ejemplo: 
                # __next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.jJsEuz > div.sc-dffc6c81-0.iwmAVw > ul > li:nth-child(4)  
                
                if len(lista_min) > 1:
                    hora = int(lista_min[0].replace('h', ''))
                    hora 
                    min = int(lista_min[1].replace('min', ''))
                    duracion = 60*hora + min
                    print(duracion)
                                
                else: 
                    duracion= int(lista_min[0].replace('min',''))
                    duracion 
                    print(duracion)

            except:
                    duracion = "Formato min incorrecto"

        except:
            duracion = "NO DATA"            
        
        lista_actores = [tupla[-1]]
        for i in range(1, 11):
            sleep(2)
            # cogemos xpath porque el selector no nos cogía bien los nombres
            try:
                try:
                    try:
                        lista_actores.append(driver.find_element("xpath", f'/html/body/div[2]/main/div/section[1]/div/section/div/div[1]/section[3]/div[2]/div[2]/div[{i}]/div[2]/a').text)
                    
                    except:
                        lista_actores.append(driver.find_element("xpath", f'/html/body/div[2]/main/div/section[1]/div/section/div/div[1]/section[4]/div[2]/div[2]/div[{i}]/div[2]/a').text)
            
                except:
                    lista_actores.append(driver.find_element("xpath", f'/html/body/div[2]/main/div/section[1]/div/section/div/div[1]/section[2]/div[2]/div[2]/div[{i}]/div[2]/a').text)
            except:
                break
        print(lista_actores)
        
        lista_tupla_actores.append(tuple(lista_actores))      
        #[("Drama","Movie","The Drop",2006,6,"tt0340084")
        #id	Nombre peli	tipo	genero	año	mes	director	director2	actor1	actorx	guionista1	guionista2	guionistas x	argumento	duracion

        #Lista de cada pelicula 
        lista_pelicula = [tupla[-1], tupla[2], tupla[1], tupla [0], tupla[3], tupla[4], imdb_director, lista_actores, guionistas, imdb_argumento, duracion]
        
        #lista  final todas las peliculas
        lista_peliculas.append(tuple(lista_pelicula))

        #lista puntuaciones imdb (id, nombre pelicula, puntos imdb, año)
        lista_imdb.append((tupla[-1], tupla[2], imdb_puntos, tupla[3]))

        print(lista_imdb)

        driver.back()

    driver.close()
    return lista_peliculas, lista_imdb, lista_tupla_actores

In [ ]:
#archivo csv lisat peliculas de la api
lista_csv_1990 = from_csv_to_lists('peliculas_1990.csv')

#pasamos lista a imdb y sacamos toda la informacion de imdb (datos peliculas, lista de actores y puntuacion imdb)
# ya para pasar a csv y bbdd --> lista_pelis --> id, nombre peli, tipo,	genero,	año, mes, director, actores, guionistas, argumento, duracion
# para pasar a rotten (google) --> lista_puntos_imdb --> id, nombre pelicula, puntos imdb, año
# para eliminar duplicados y pasar a imdb para buscar datos actores --> lista_actores_imdb --> (id, actor1, actor2, ..)

lista_peliculas_1990, lista_puntos_imdb_1990, lista_actores_imdb_1990 = datos_imdb(lista_csv_1990)

In [ ]:
#pasar lista peliculas con todos los datos a csv
df_peliculas_1990 = pd.DataFrame(lista_peliculas_1990)
df_peliculas_1990.to_csv('lista_peliculas_1990.csv', index=False)

In [ ]:
## ROTTENTOMATOES

#listas pruebas
#lista = [('tt1856101', 'Blade Runner 2049', '8,0', 2017)]
#lista_imdb = [('tt0340084', 'The Drop', '2,7', 2006), ('tt5142104', 'Geocatching', 'NO DATA', 2015), ('tt1856101', 'Blade Runner 2049', '8,0', 2017), ('tt0106977', 'The Fugitive', '7,8', 1993)]

def puntuacion(lista_puntos_pelis):
    """
    Esta función toma una lista de tuplas de peliculas (id pelicula, titulo, puntuacion imdb, año) y crea una nueva lista de tuplas con las puntuaciones de imdb y rotten tomatoes.
    
    Args:
    - lista__puntos_pelis: lista de tuplas de tuplas de peliculas (id pelicula, titulo, puntuacion imdb, año).

    Returns:
    - lista_puntuacion: lista de tuplas de puntuaciones de peliculas (id pelicula, nombre pelicula, puntuaion imdb, puntuacion rotten tomatoes). 
    """

    #Cambiar 'Firefox' por el navegador a usar
    driver = webdriver.Firefox()

    lista_puntuacion = []

    driver.get("https://www.google.es/")

    driver.maximize_window()

    for peli in lista_puntos_pelis:

        sleep(1)
        #cookies
        try:
            driver.find_element("css selector", "#L2AGLb > div").click()
        except:
            pass 
      
        driver.find_element("css selector", "#APjFqb").send_keys(f'"{peli[1]}" "{peli[3]}" Rotten Tomatoes', Keys.ENTER)
        
        #para las peliculas que no aparecn en rottten
        try:
            sleep(2)
            valoracion = driver.find_element("css selector","div.hlcw0c:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > span:nth-child(2)").text
            puntos_rotten = int(valoracion.split(':')[1].replace('%', ''))

        except:
            puntos_rotten = "NO DATA"
            lista_puntuacion.append((peli[0], peli[1], peli[2], puntos_rotten))
            driver.back()

            continue
                
        lista_puntuacion.append((peli[0], peli[1], peli[2], puntos_rotten))
        driver.back()
    driver.close()
    return lista_puntuacion    

In [ ]:
#lista de puntos imdb a rotten y devuelve una lisat final con las puntuaciones --> id, nombre peli, puntos imdb, puntos rotten --> a df y a csv
puntuaciones_1990 = puntuacion(lista_puntos_imdb_1990)
df_puntuaciones_1990 = pd.DataFrame(puntuaciones_1990)
df_puntuaciones_1990.to_csv('puntuaciones_1990.csv', index=False)

df_puntuaciones_1990

In [ ]:
# ACTORES únicos:
def actores_unicos(lista_tupla_actores):
    """
    Esta función toma una lista de tuplas de (id pelicula y 10 actores principales de la pelicula) y devuelve una lista de actores únicos.
    
    Args:
    - lista_tupla_actores: lista de tuplas de 10 actores principlaes de una peli (id pelicula, actor1-actor10) depues de pasar por la funcion datos_imdb().

    Returns:
    - set(actores): un ser de todos los actores únicos. 
    """
    actores=[]
    for tupla in lista_tupla_actores:
        for actor in tupla[1:]:
            actores.append(actor)
    return set(actores)

In [ ]:
#Informacion actores
def datos_actores(lista_actores):
    """
    Esta función toma una lista de actores y busca información de cada actor en imdb.

    Args:
    - lista_actores: lista de actores (lista de actores que viene de pasar por la funcion datos_imdb() y actores_unicos()

    Returns:
    - detalles_actores: lista de datos de cada actor en una tupla (nombre actor, año nacimiento, conocido por, que hace, premios oscar)
    """

    driver = webdriver.Chrome()

    driver.get("https://www.imdb.com/")

    driver.maximize_window()

    sleep(1)
    
    detalles_actores = []
    detalles_actor = []

    for actor in lista_actores:
        
        #Escribe el nombre del actor en el buscador
        driver.find_element("css selector", "#suggestion-search").send_keys(actor, Keys.ENTER)
        
        sleep(1)

        driver.find_element("css selector", '#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-383f2ac5-0.bfcGjo > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3) > div.sc-17bafbdb-2.ffAEHI > ul > li:nth-child(1) > div.ipc-metadata-list-summary-item__c > div > a').click()
        

        sleep(1)
        try:
            que_hace = driver.find_element("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.jJsEuz > div > ul').text
            que_hace = que_hace.split('\n')
        except:
            que_hace = 'No data'  

        sleep(1)
        try:
            conocido = driver.find_element("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > div > section > div > div.sc-9178d6fe-1.kFxVZc.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-a6d4b6c0-0.jGufEe').text
            conocido = conocido.split('\n')
        except:
            conocido = 'No data'

        sleep(1)
        try:
            año = driver.find_element("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-4.ecgcFy > div.sc-e226b0e3-6.hfusNC > div.sc-e226b0e3-11.jSQoAO > section > aside > div > span:nth-child(2)').text.split(' ')[-1]
        except:
            año = 'No data'
        

        sleep(1)
        try:
            premios = driver.find_element("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > div > section > div > div.sc-9178d6fe-1.kFxVZc.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3) > div > ul > li > a.ipc-metadata-list-item__label.ipc-metadata-list-item__label--link').text
            if premios == 'Premios':
                premios = 'Sin Oscar'
        except:
            premios = 'Sin oscar'
        
        #nombre actor, año nacimiento, conocido por, que hace, premios oscar
        detalles_actor = (actor, año, conocido, que_hace, premios)

        detalles_actores.append(detalles_actor)
        
    return detalles_actores    

In [ ]:
#lista de los datos de los actores a df y de df a csv
actores_unicos_1990 = actores_unicos(lista_actores_imdb_1990)
lista_datos_actores_1990 = datos_actores(actores_unicos_1990)
df_datos_actores_1990 = pd.DataFrame(lista_datos_actores_1990)
df_datos_actores_1990.to_csv('lista_actores_1990.csv', index=False)

df_datos_actores_1990